# Import data

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import math
%matplotlib inline

found as csv on http://www.football-data.co.uk/germanym.php

In [2]:
game_mat_df = pd.read_csv('D1_2017_2018.csv', usecols=['Date','HomeTeam','AwayTeam','FTHG','FTAG'])
goals_df = game_mat_df

All team names need to be converted to a team index for numerical handling

In [5]:
team_names = game_mat_df['HomeTeam'].unique()
team_index = list(range(1,len(team_names)+1))
teams_dict = dict(zip(team_names,team_index))

def team_lookup(name):
    index = teams_dict[name]
    return index

In [6]:
goals_ind_df = goals_df.copy()
goals_ind_df['HomeTeam'] = goals_df['HomeTeam'].apply(team_lookup)
goals_ind_df['AwayTeam'] = goals_df['AwayTeam'].apply(team_lookup)

## Probabilistic model

The dataframe contains now the number of goals scored by the home (FTHG) and away team (FTAG) for the gth (index) match

In [25]:
num_teams = 18
goals_ind_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,18/08/17,1,14,3,1
1,19/08/17,2,11,1,0
2,19/08/17,3,15,2,0
3,19/08/17,4,16,1,0
4,19/08/17,5,17,0,1


FTHG and FTAG are assumed to be realizations of conditionally independent random variables Y_{g,H} and Y_{g,A}

In [26]:
with pm.Model() as model:
    mu_att = pm.Normal('mu_att', mu=0, sd=1/math.sqrt(0.0001))
    mu_defe = pm.Normal('mu_defe', mu=0, sd=1/math.sqrt(0.0001))
    tau_att = pm.Normal('tau_att', mu=0.1, tau=0.1)
    tau_defe = pm.Normal('tau_defe', mu=0.1, tau=0.1)
    home = pm.Normal('home', mu=0, sd=1/math.sqrt(0.0001))
    
    att = pm.Normal('att', mu=mu_att, tau=tau_att, size=num_teams)
    defe = pm.Normal('defe', mu=mu_defe, tau=tau_defe)
    
    theta_H = (home + att + defe)
    theta_A = (att + defe)
    
    y_H = pm.Poisson('y_H', theta_H, observed=goals_ind_df['FTHG'])
    y_A = pm.Poisson('y_A', theta_A, observed=goals_ind_df['FTAG'])

TypeError: __init__() got an unexpected keyword argument 'size'